In [1]:
!kaggle datasets download -d andradaolteanu/gtzan-dataset-music-genre-classification

Dataset URL: https://www.kaggle.com/datasets/andradaolteanu/gtzan-dataset-music-genre-classification
License(s): other
 99%|██████████████████████████████████████▌| 1.20G/1.21G [00:09<00:00, 143MB/s]
100%|███████████████████████████████████████| 1.21G/1.21G [00:09<00:00, 142MB/s]


In [2]:
!unzip -qq gtzan-dataset-music-genre-classification.zip

In [3]:
!rm -r Data/genres_original/disco
!rm -r Data/genres_original/blues

In [5]:
import numpy as np
import tensorflow as tf
import random
import os
import torch

def set_random_seeds(seed_value):
    np.random.seed(seed_value)
    tf.random.set_seed(seed_value)
    random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)

seed_value = 42
set_random_seeds(seed_value)

In [6]:
SOURCE_PATH = '/kaggle/working/Data/genres_original'

JSON_PATH = 'data.json'

sr = 22050

TOTAL_SAMPLES = 29 * sr

NUM_SLICES = 10
SAMPLES_PER_SLICE = int(TOTAL_SAMPLES / NUM_SLICES)

n_fft = 1279
hop_length = 260
n_mels = 40
n_mfcc = 13
win_length = 1024
window = 'hamming'
dct_type = 2

In [14]:
from tqdm import tqdm
import os
import librosa
import json
import gc


genre_dict = {
      0 : "metal",
      1 : "jazz",
      2 : "rock",
      3 : "country",
      4 : "hiphop",
      5 : "classical",
      6 : "reggae",
      7 : "pop",
      }
reverse_genre_dict = {v: k for k, v in genre_dict.items()}

def preprocess_data(source_path, json_path):

    total_files = sum([len(files) for _, _, files in os.walk(source_path)])

    progress_bar = tqdm(total=total_files, desc="Processing", unit="file")
    leng = int((22050 * 3) / 2)

    splits = 5
    mfcc = np.zeros((total_files * splits, 640, 128))
    labels = np.zeros((total_files * splits))

    counter = 0
    for i, (dirpath, dirnames, filenames) in enumerate(tqdm(os.walk(source_path))):
        for file in filenames:
            try:
                progress_bar.set_description(f"Processing: {file}")

                y, sr = librosa.load(os.path.join(dirpath, file))

                k = leng
                for p in range(splits):
                  D = librosa.stft(y[k-leng:k], n_fft=n_fft, hop_length=hop_length, win_length=win_length, window=window)
                  DB = librosa.amplitude_to_db(abs(D))
                  label = reverse_genre_dict[file.split('.')[0]]

                  print(file + " " + str(p) + " " + str(DB.shape) + " " + str(label))

                  k = k + leng

                  labels[counter] = label
                  mfcc[counter] = DB
                  counter = counter + 1

                progress_bar.update(1)
            except Exception as e:
                print(f"Issue with {file}: {e}")

    progress_bar.close()

    del y, sr, D, DB
    gc.collect()

    return mfcc, labels

In [15]:
def load_data(json_path):

    with open(json_path, 'r') as f:
        data = json.load(f)
    f.close()

    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    return X, y

In [16]:
def prepare_datasets(inputs, targets, split_size):

    inputs_train, inputs_val, targets_train, targets_val = train_test_split(inputs, targets, test_size=split_size)
    inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs_train, targets_train, test_size=split_size)

    return inputs_train, inputs_val, inputs_test, targets_train, targets_val, targets_test

In [17]:
inputs, targets = preprocess_data(source_path=SOURCE_PATH, json_path=JSON_PATH)

Processing: pop.00067.wav:   0%|          | 4/800 [00:00<00:43, 18.15file/s]

pop.00045.wav 0 (640, 128) 7
pop.00045.wav 1 (640, 128) 7
pop.00045.wav 2 (640, 128) 7
pop.00045.wav 3 (640, 128) 7
pop.00045.wav 4 (640, 128) 7
pop.00033.wav 0 (640, 128) 7
pop.00033.wav 1 (640, 128) 7
pop.00033.wav 2 (640, 128) 7
pop.00033.wav 3 (640, 128) 7
pop.00033.wav 4 (640, 128) 7
pop.00041.wav 0 (640, 128) 7
pop.00041.wav 1 (640, 128) 7
pop.00041.wav 2 (640, 128) 7
pop.00041.wav 3 (640, 128) 7
pop.00041.wav 4 (640, 128) 7
pop.00096.wav 0 (640, 128) 7
pop.00096.wav 1 (640, 128) 7
pop.00096.wav 2 (640, 128) 7
pop.00096.wav 3 (640, 128) 7
pop.00096.wav 4 (640, 128) 7


Processing: pop.00039.wav:   1%|          | 8/800 [00:00<00:41, 18.94file/s]

pop.00067.wav 0 (640, 128) 7
pop.00067.wav 1 (640, 128) 7
pop.00067.wav 2 (640, 128) 7
pop.00067.wav 3 (640, 128) 7
pop.00067.wav 4 (640, 128) 7
pop.00059.wav 0 (640, 128) 7
pop.00059.wav 1 (640, 128) 7
pop.00059.wav 2 (640, 128) 7
pop.00059.wav 3 (640, 128) 7
pop.00059.wav 4 (640, 128) 7
pop.00022.wav 0 (640, 128) 7
pop.00022.wav 1 (640, 128) 7
pop.00022.wav 2 (640, 128) 7
pop.00022.wav 3 (640, 128) 7
pop.00022.wav 4 (640, 128) 7
pop.00064.wav 0 (640, 128) 7
pop.00064.wav 1 (640, 128) 7
pop.00064.wav 2 (640, 128) 7
pop.00064.wav 3 (640, 128) 7
pop.00064.wav 4 (640, 128) 7


Processing: pop.00005.wav:   1%|▏         | 11/800 [00:00<00:43, 18.15file/s]

...

rock.00066.wav 4 (640, 128) 2
rock.00087.wav 0 (640, 128) 2
rock.00087.wav 1 (640, 128) 2
rock.00087.wav 2 (640, 128) 2
rock.00087.wav 3 (640, 128) 2
rock.00087.wav 4 (640, 128) 2
rock.00078.wav 0 (640, 128) 2
rock.00078.wav 1 (640, 128) 2
rock.00078.wav 2 (640, 128) 2
rock.00078.wav 3 (640, 128) 2
rock.00078.wav 4 (640, 128) 2
rock.00046.wav 0 (640, 128) 2
rock.00046.wav 1 (640, 128) 2
rock.00046.wav 2 (640, 128) 2
rock.00046.wav 3 (640, 128) 2
rock.00046.wav 4 (640, 128) 2
rock.00051.wav 0 (640, 128) 2
rock.00051.wav 1 (640, 128) 2
rock.00051.wav 2 (640, 128) 2


Processing: rock.00074.wav: 100%|█████████▉| 796/800 [00:47<00:00, 17.78file/s]

rock.00051.wav 3 (640, 128) 2
rock.00051.wav 4 (640, 128) 2
rock.00086.wav 0 (640, 128) 2
rock.00086.wav 1 (640, 128) 2
rock.00086.wav 2 (640, 128) 2
rock.00086.wav 3 (640, 128) 2
rock.00086.wav 4 (640, 128) 2
rock.00027.wav 0 (640, 128) 2
rock.00027.wav 1 (640, 128) 2
rock.00027.wav 2 (640, 128) 2
rock.00027.wav 3 (640, 128) 2
rock.00027.wav 4 (640, 128) 2
rock.00052.wav 0 (640, 128) 2
rock.00052.wav 1 (640, 128) 2
rock.00052.wav 2 (640, 128) 2
rock.00052.wav 3 (640, 128) 2
rock.00052.wav 4 (640, 128) 2
rock.00074.wav 0 (640, 128) 2


Processing: rock.00008.wav: 100%|█████████▉| 799/800 [00:47<00:00, 17.33file/s]
9it [00:47,  5.28s/it]
Processing: rock.00008.wav: 100%|█████████▉| 799/800 [00:47<00:00, 16.83file/s]


rock.00074.wav 1 (640, 128) 2
rock.00074.wav 2 (640, 128) 2
rock.00074.wav 3 (640, 128) 2
rock.00074.wav 4 (640, 128) 2
rock.00043.wav 0 (640, 128) 2
rock.00043.wav 1 (640, 128) 2
rock.00043.wav 2 (640, 128) 2
rock.00043.wav 3 (640, 128) 2
rock.00043.wav 4 (640, 128) 2
rock.00008.wav 0 (640, 128) 2
rock.00008.wav 1 (640, 128) 2
rock.00008.wav 2 (640, 128) 2
rock.00008.wav 3 (640, 128) 2
rock.00008.wav 4 (640, 128) 2


In [18]:
!git clone https://github.com/YuanGongND/ast.git

Cloning into 'ast'...
remote: Enumerating objects: 649, done.
remote: Counting objects: 100% (209/209), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 649 (delta 172), reused 159 (delta 159), pack-reused 440
Receiving objects: 100% (649/649), 2.41 MiB | 26.21 MiB/s, done.
Resolving deltas: 100% (360/360), done.


In [19]:
from sklearn.model_selection import train_test_split

Xtrain, Xval, Xtest, ytrain, yval, ytest = prepare_datasets(inputs, targets, 0.2)

In [20]:
print(Xtrain.shape)

print(Xtrain.shape)
print(ytrain.shape)

print(Xtest.shape)
print(ytest.shape)

print(Xval.shape)
print(yval.shape)

(2560, 640, 128)
(2560, 640, 128)
(2560,)
(640, 640, 128)
(640,)
(800, 640, 128)
(800,)


In [21]:
Xtrain_tensor = torch.tensor(Xtrain, dtype=torch.float32)
ytrain_tensor = torch.tensor(ytrain, dtype=torch.long)
Xval_tensor = torch.tensor(Xval, dtype=torch.float32)
yval_tensor = torch.tensor(yval, dtype=torch.long)
Xtest_tensor = torch.tensor(Xtest, dtype=torch.float32)
ytest_tensor = torch.tensor(ytest, dtype=torch.long)

In [22]:
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

train_dataset = TensorDataset(Xtrain_tensor, ytrain_tensor)
val_dataset = TensorDataset(Xval_tensor, yval_tensor)
test_dataset = TensorDataset(Xtest_tensor, ytest_tensor)

In [23]:
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8)
test_loader = DataLoader(test_dataset, batch_size=8)

In [24]:
import sys
# if 'google.colab' in sys.modules:
#     print('Running in Colab.')
#     !git clone https://github.com/YuanGongND/ast
#     sys.path.append('./ast')

! pip install timm==0.4.5
! pip install wget
import os, csv, argparse, wget
os.environ['TORCH_HOME'] = '/kaggle/working/ast/pretrained_models'
if os.path.exists('/kaggle/working/ast/pretrained_models') == False:
  os.mkdir('/kaggle/working/ast/pretrained_models')
import torch, torchaudio, timm
import numpy as np
from torch.cuda.amp import autocast
import IPython

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 9.6 MB/s eta 0:00:00
  Attempting uninstall: timm
    Found existing installation: timm 0.9.16
    Uninstalling timm-0.9.16:
      Successfully uninstalled timm-0.9.16
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9655 sha256=76f6e483ccc16ff7029d7261974e532f0c801378ec4c1980627d1b5e821af5ac
  Stored in directory: /root/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


In [25]:
%cd /kaggle/working/ast/src/models

/kaggle/working/ast/src/models


In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [28]:
from ast_models import ASTModel

ast_model = ASTModel(label_dim=8, fstride=10, tstride=10, input_fdim=640, input_tdim=128, imagenet_pretrain=True, audioset_pretrain=False, model_size='tiny224').to(device)

---------------AST Model Summary---------------
ImageNet pretraining: True, AudioSet pretraining: False
frequncey stride=10, time stride=10
number of patches=756


In [105]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PRCNN(nn.Module):
    def __init__(self, input_shape, nb_classes):
        super(PRCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=input_shape[0], out_channels=16, kernel_size=(3,1))
        self.pool11 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2,2))
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=(3,1))
        self.pool12 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2,2))
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,1))
        self.pool13 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2,2))
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,1))
        self.pool14 = nn.MaxPool2d(kernel_size=(4, 4), stride=(4,4))
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=(3,1))
        self.pool15 = nn.MaxPool2d(kernel_size=(4, 4), stride=(4,4))

        self.flatten1 = nn.Flatten()

        # BGRU-RNN block
        self.pool21 = nn.MaxPool2d(kernel_size=(1,2), stride=(1,2))
        self.bigru_rnn = nn.GRU(input_size=40960, hidden_size=128, batch_first=True, bidirectional=True)
        self.fc_gru = nn.Linear(512, 256)  # nowa warstwa liniowa

        # Softmax Output
        self.fc = nn.Linear(256, nb_classes)

    def forward(self, x):
        x_cnn = F.relu(self.conv1(x))
        x_cnn = self.pool11(x_cnn)
        x_cnn = F.relu(self.conv2(x_cnn))
        x_cnn = self.pool12(x_cnn)
        x_cnn = F.relu(self.conv3(x_cnn))
        x_cnn = self.pool13(x_cnn)
        x_cnn = F.relu(self.conv4(x_cnn))
        x_cnn = self.pool14(x_cnn)
        x_cnn = F.relu(self.conv5(x_cnn))
        x_cnn = self.pool15(x_cnn)

        x_cnn = self.flatten1(x_cnn)
        
#         print("CNN shape: ", x_cnn.shape)

        # print(x.shape)
        z = self.pool21(x)
        batch_size = x.size(0)
        z = self.flatten1(z)

        # GRU layer
        x_gru, _ = self.bigru_rnn(z)
        
#         print("RNN shape: ", x_gru.shape)


        x_concatenated = torch.cat((x_cnn, x_gru), dim=1)

        x_concatenated = self.fc_gru(x_concatenated)

        output = self.fc(x_concatenated)
        return output

    def adjust_input_size(self, x):
        x = self.linear(x)
        return x

    def linear(self, x):
        return F.relu(nn.Linear(x.size(-1), 16)(x))

In [30]:
class HybridModel(nn.Module):
    def __init__(self, ast_model, prcnn_model, num_classes):
        super(HybridModel, self).__init__()
        self.ast_model = ast_model
        self.prcnn_model = prcnn_model
        self.fc = nn.Linear(16, num_classes)

    def forward(self, x):
        x_prcnn_input = x.unsqueeze(1)

        x_ast = self.ast_model(x)
        x_prcnn = self.prcnn_model(x_prcnn_input)

        x_concat = torch.cat((x_ast, x_prcnn), dim=1)

        output = self.fc(x_concat)
        return output

In [31]:
input_shape = (1, Xtrain.shape[1], Xtrain.shape[2])
prcnn_model = PRCNN(input_shape, 8)

In [32]:
hybrid_model = HybridModel(ast_model, prcnn_model, num_classes=8)

In [33]:
hybrid_model.to(device)

HybridModel(
  (ast_model): ASTModel(
    (v): DistilledVisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(1, 192, kernel_size=(16, 16), stride=(10, 10))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (blocks): ModuleList(
        (0-11): 12 x Block(
          (norm1): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=192, out_features=576, bias=True)
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=192, out_features=192, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (drop_path): Identity()
          (norm2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=192, out_features=768, bias=True)
            (act): GELU(approximate='none')
            (fc2): Linear(in_features=768, out_features=192, bias=True)
            (drop): Drop

In [34]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(hybrid_model.parameters(), lr=0.001)

def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=5):
    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        correct = 0
        total = 0

        progress_bar = tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}', leave=False)

        for inputs, labels in progress_bar:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

            progress_bar.set_postfix({'train_loss': train_loss / total, 'train_acc': 100 * correct / total})

        val_loss, val_acc = evaluate_model(model, val_loader, criterion)

        print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss/len(train_loader.dataset):.4f}, Train Acc: {100*correct/total:.2f}%, Val Loss: {val_loss:.4f}, Val Acc: {val_acc:.2f}%')

def evaluate_model(model, val_loader, criterion):
    model.eval()
    val_loss = 0.0
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * inputs.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    val_loss /= len(val_loader.dataset)
    val_acc = 100 * correct / total
    return val_loss, val_acc

In [35]:
num_epochs = 30
train_model(hybrid_model, train_loader, val_loader, criterion, optimizer, num_epochs)

Epoch [1/30], Train Loss: 1.7232, Train Acc: 34.34%, Val Loss: 1.3540, Val Acc: 49.88%


Epoch [2/30], Train Loss: 1.3530, Train Acc: 48.71%, Val Loss: 1.1581, Val Acc: 56.88%


Epoch [3/30], Train Loss: 1.2064, Train Acc: 55.59%, Val Loss: 1.1072, Val Acc: 60.38%


Epoch [4/30], Train Loss: 1.1129, Train Acc: 57.19%, Val Loss: 0.9853, Val Acc: 64.88%


Epoch [5/30], Train Loss: 1.0189, Train Acc: 61.91%, Val Loss: 0.9811, Val Acc: 62.50%


...

Epoch [26/30], Train Loss: 0.2055, Train Acc: 92.66%, Val Loss: 1.3439, Val Acc: 69.38%


Epoch [27/30], Train Loss: 0.1757, Train Acc: 93.98%, Val Loss: 1.3564, Val Acc: 70.88%


Epoch [28/30], Train Loss: 0.1878, Train Acc: 93.09%, Val Loss: 1.2512, Val Acc: 68.88%


Epoch [29/30], Train Loss: 0.1398, Train Acc: 95.59%, Val Loss: 1.4746, Val Acc: 69.12%


Epoch [30/30], Train Loss: 0.1545, Train Acc: 94.65%, Val Loss: 1.4008, Val Acc: 69.25%


In [36]:
def test_model(model, test_loader):
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)

    test_acc = 100 * correct / total
    print(f'Test Accuracy: {test_acc:.2f}%')

test_model(hybrid_model, test_loader)

Test Accuracy: 70.78%
